In [24]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0   7416      0 --:--:-- --:--:-- --:--:--  7416
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5452    0  5452    0     0  50018      0 --:--:-- --:--:-- --:--:-- 50018


In [23]:
import pandas as pd
import torch
from torch import optim, nn
from torchvision import models, transforms
df = pd.read_csv('csvs/val_final.csv')

In [ ]:
# from pycocotools.coco import COCO
# import numpy as np
# import skimage.io as io
# import matplotlib.pyplot as plt
# import pylab
# pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
# image_urls=df.coco_url.to_list()
# images=[io.imread(image_url) for image_url in image_urls[:100]] 

In [ ]:
# with open("images.txt", "w") as output:
#     output.write(str('\n'.join(set(image_urls))))

In [ ]:
# print(len(image_urls))
# print(len(set(image_urls)))

In [ ]:
# %cd image
# !wget -i ../images.txt

In [ ]:
# from skimage.io import imread_collection

# #your path 
# img_dir = '/home/development/sakharamg/Test/image/*.jpg'

# #creating a collection with the available images
# image_collection = imread_collection(img_dir)

In [ ]:
# model = models.vgg16(pretrained=True)


In [ ]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
		# Extract VGG-16 Feature Layers
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
		# Extract VGG-16 Average Pooling Layer
    self.pooling = model.avgpool
		# Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
		# Extract the first part of fully-connected layer from VGG16
    self.fc = model.classifier[0]
  
  def forward(self, x):
		# It will take the input 'x' until it returns the feature vector called 'out'
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out) 
    return out 

# Initialize the model
resnet = models.resnet152(pretrained=True)
modules = list(resnet.children())[:-1] # delete the last fc layer.
resnet = nn.Sequential(*modules)
### Now set requires_grad to false
for param in resnet.parameters():
  param.requires_grad = False

# Change the device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device="cpu"
# new_model= nn.DataParallel(resnet)
new_model = resnet.to(device)

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(512),
    transforms.Resize(224), #448
    transforms.ToTensor()                              
    ])
    images = {}
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = transform(img)
        # print(img.shape)
        img = img.reshape(1, 3, 224, 224) 
        img = img.to(device)
        if img is not None:
            images[int(filename.split('.')[0])] = img
    return images

In [ ]:
images_collection = load_images_from_folder('image/')

In [ ]:
# import os
# image_dict={}
# for file in images_collection.files:
#     image_dict[int(os.path.basename(file).split('.')[0])]=images_collection.load_func(file)

In [ ]:
features=[]
for id in df.image_id:
    # print(id)
    with torch.no_grad():
		# Extract the feature from the image
        feature = new_model(images_collection[id])
	    # Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))
# Convert to NumPy Array

    # image_matrix_list.append(images_collection[id])

In [ ]:
# import numpy as np
# image_matrix_data=np.concatenate( image_matrix_list )

In [ ]:
import numpy as np
features = np.array(features)

In [ ]:
features.shape

In [ ]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-mpnet-base-v2')
model = SentenceTransformer('clip-ViT-B-32')

In [ ]:
caption_pos_encodings = model.encode(df.caption.to_list())
caption_neg_encodings = model.encode(df.negative_captions.to_list())

In [ ]:

row_sums = features.sum(axis=1)
features_norm = features / row_sums[:, np.newaxis]


row_sums = caption_pos_encodings.sum(axis=1)
caption_pos_encodings_norm = caption_pos_encodings / row_sums[:, np.newaxis]



row_sums = caption_neg_encodings.sum(axis=1)
caption_neg_encodings_norm = caption_neg_encodings / row_sums[:, np.newaxis]



X_correct=np.concatenate((features_norm, caption_pos_encodings_norm), axis=1)
X_incorrect=np.concatenate((features_norm, caption_neg_encodings_norm), axis=1)

In [ ]:
y_correct=np.ones(len(X_correct))
y_incorrect=np.zeros(len(X_correct))

In [ ]:
X_data=np.concatenate((X_correct, X_incorrect), axis=0)
y_data=np.concatenate((y_correct, y_incorrect), axis=0)




In [ ]:
def unison_shuffled_copies(a, b):
    seed=0
    np.random.seed(seed)
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
# X_shuffled,y_shuffled=unison_shuffled_copies(X_data,y_data)
from sklearn.utils import shuffle
X_shuffled, y_shuffled = shuffle(X_data, y_data, random_state=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.10, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import pickle
import math
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle

In [ ]:
X_train.shape[1]-2048

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate, LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Bidirectional


In [ ]:
image_representation = Input(shape=(2048,))
image_representation1=Dense(2048, activation='relu')(image_representation)
image_representation2=Dense(1024, activation='relu')(image_representation1)
image_representation3=Dense(768, activation='relu')(image_representation2)
image_representation3=Dense(512, activation='relu')(image_representation3)
sentence_representation = Input(shape=(512,))
sentence_representation1=Dense(512, activation='relu')(sentence_representation)
sentence_representation2=Dense(512, activation='relu')(sentence_representation1)
sentence_representation3=Dense(512, activation='relu')(sentence_representation2)

merged = Concatenate()([image_representation3,sentence_representation3])
merged=Dense(1400, activation='relu')(merged)
merged=Dense(1400, activation='relu')(merged)
merged=Dense(700, activation='relu')(merged)
merged=Dense(100, activation='relu')(merged)
merged=Dense(2, activation='softmax')(merged)

In [ ]:
enc_dec_model = Model([image_representation,sentence_representation], Activation('softmax')(merged))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
            optimizer=Adam(1e-3),
            metrics=['accuracy'])
enc_dec_model.summary()

In [ ]:
model_results = enc_dec_model.fit([X_train[:,:2048],X_train[:,2048:]],y_train, batch_size=30, epochs=100)

In [ ]:
# dropout=0.2
model1 = Sequential()
model1.add(Dense(4864, input_dim=len(np.array(X_train)[0]), activation='relu'))
# layers.Dropout(dropout, input_shape=(4864,)),
model1.add(Dense(4864, activation='relu'))
# layers.Dropout(dropout, input_shape=(4864,)),
model1.add(Dense(2400, activation='relu'))
# layers.Dropout(dropout, input_shape=(2400,)),
model1.add(Dense(2400, activation='relu'))
# layers.Dropout(dropout, input_shape=(2400,)),
model1.add(Dense(1200, activation='relu'))
# layers.Dropout(dropout, input_shape=(1200,)),
model1.add(Dense(1200, activation='relu'))
# layers.Dropout(dropout, input_shape=(1200,)),
model1.add(Dense(600, activation='relu'))
# layers.Dropout(dropout, input_shape=(600,)),
model1.add(Dense(300, activation='relu'))
# layers.Dropout(dropout, input_shape=(300,)),
model1.add(Dense(150, activation='relu'))
# layers.Dropout(dropout, input_shape=(150,)),
model1.add(Dense(2, activation='softmax'))

In [ ]:
model1.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics =['accuracy'])

In [ ]:
model1.fit(X_train, y_train,validation_split = 0.1, epochs = 10)

In [ ]:
# score, acc = model1.evaluate(X_test, y_test)
# print('Test loss:', score)
# print('Test accuracy:', acc)

In [19]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    max_samples=50,
    shuffle=True,
)

Found annotations at '/home/development/sakharamg/fiftyone/coco-2017/raw/instances_val2017.json'
49 images found; downloading the remaining 1
 100% |██████████████████████| 1/1 [1.4s elapsed, 0s remaining, 0.7 images/s] 
Writing annotations for 50 downloaded samples to '/home/development/sakharamg/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/home/development/sakharamg/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████████| 50/50 [276.3ms elapsed, 0s remaining, 182.3 samples/s]      
Dataset 'coco-2017-validation-50' created


In [21]:
dataset

Name:        coco-2017-validation-50
Media type:  image
Num samples: 50
Persistent:  False
Tags:        ['validation']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)